In [1]:
import glob
import os
import random
import shutil
import sys
from pathlib import Path

import numpy as np
import scipy.io
import torch

current_dir = os.path.dirname(os.path.abspath("__file__"))  # 当前目录
parent_dir = os.path.abspath(os.path.join(current_dir, ".."))  # 上一级目录
sys.path.append(parent_dir)
grandparent_dir = os.path.abspath(os.path.join(parent_dir, ".."))  # 上两级目录
sys.path.append(grandparent_dir)

import data_function
import metrics
import tools
import utils
from ex_main.model import *
from ex_main.train_dataloader import getData
from utils.config_plot import *

In [2]:
ppath = "/home/hy/project/reid/ex_main/"

# Config
config_file_path = os.path.join(ppath, "config.py")
config = utils.common.read_config_file(config_file_path)

# Initialize a logger tool
logger = utils.logger.Logger(config.outputs_path)
logger.info("#" * 50)
logger.info(f"Task: {config.taskname}")
logger.info(f"Using device: {config.device}")
logger.info(f"Using data type: {config.dtype}")

# Set environment
random.seed(config.seed)
np.random.seed(config.seed)
torch.manual_seed(config.seed)
torch.cuda.manual_seed(config.seed)
torch.cuda.manual_seed_all(config.seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False  # The result cannot be reproduced when True

2024-03-14 21:38:14 INFO ##################################################
2024-03-14 21:38:14 INFO Task: ReID_Task
2024-03-14 21:38:14 INFO Using device: cuda
2024-03-14 21:38:14 INFO Using data type: torch.float32


In [3]:
config.batch_size = 1
config.test_batch_size = 1

train_loader, query_loader, gallery_loader, num_classes = getData(config=config)

In [4]:
import cv2
import torchvision.transforms as T
from tqdm import tqdm

IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]


def parse_RBG2Gray(img):
    transform = T.Grayscale(num_output_channels=3)
    img = transform(img)
    return img


def transformTensor2npRGB(img):
    _img = torch.zeros_like(img)
    for index, (t, m, s) in enumerate(zip(img, IMAGENET_MEAN, IMAGENET_STD)):
        _img[index] = t.mul(s).add(m).clamp(0, 1)
    img_np = np.uint8(np.floor(_img.numpy() * 255))
    img_np = img_np.transpose((1, 2, 0))  # (c, h, w) -> (h, w, c)
    return img_np


def generate_data(dataset, save_path):
    for batch_idx, data in enumerate(tqdm(dataset)):
        img, pid, camid, img_path = data

        # Parse image
        pGrayimg = parse_RBG2Gray(img)

        # Get image name
        imname = os.path.basename(os.path.splitext(img_path)[0])

        utils.common.mkdir_if_missing(save_path)
        path = os.path.join(save_path, imname) + ".pt"
        torch.save(pGrayimg, path)

        # Get RGB image
        img_np = transformTensor2npRGB(img)
        pGrayimg_np = transformTensor2npRGB(pGrayimg)

        # fig = plt.figure()
        # plt.imshow(img_np)
        # fig = plt.figure()
        # plt.imshow(pGrayimg_np, cmap="gray")
        # return 
    return

In [5]:
# # Set up the outputs directory
# gdpath = "/home/hy/project/reid/data/Gray-Market-1501-v15.09.15"
# if os.path.exists(gdpath):
#     shutil.rmtree(gdpath)
# os.makedirs(gdpath)

# # Data set
# img_transforms = T.Compose(
#     [
#         T.Resize((config.img_height, config.img_width), interpolation=3),
#         T.ToTensor(),
#         T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
#     ]
# )
# dataset = data_function.datasets.Market1501(root=config.dataset_path)
# train_set = data_function.ImageDataset(dataset.train, img_transforms)
# query_set = data_function.ImageDataset(dataset.query, img_transforms)
# gallery_set = data_function.ImageDataset(dataset.gallery, img_transforms)

# # Generate data
# gdsave_path = os.path.join(
#     gdpath,
#     "bounding_box_train",
# )
# generate_data(train_set, save_path=gdsave_path)

# gdsave_path = os.path.join(
#     gdpath,
#     "query",
# )
# generate_data(query_set, save_path=gdsave_path)
# gdsave_path = os.path.join(
#     gdpath,
#     "bounding_box_test",
# )
# generate_data(gallery_set, save_path=gdsave_path)

In [6]:
torch.load("/home/hy/project/reid/data/Gray-Market-1501-v15.09.15/bounding_box_train/0002_c1s1_000451_03.pt").shape

torch.Size([3, 256, 128])

In [7]:
# Set up the outputs directory
gdpath = "/home/hy/project/reid/data/Gray-Market-1501-v15.09.15-test"
if os.path.exists(gdpath):
    shutil.rmtree(gdpath)
os.makedirs(gdpath)

# Data set
img_transforms = T.Compose(
    [
        T.Resize((config.img_height, config.img_width), interpolation=3),
        T.ToTensor(),
        T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
    ]
)
dataset = data_function.datasets.Market1501(root=config.dataset_path)
train_set = data_function.ImageDataset(dataset.train, img_transforms)
query_set = data_function.ImageDataset(dataset.query, img_transforms)
gallery_set = data_function.ImageDataset(dataset.gallery, img_transforms)

# Generate data
gdsave_path = os.path.join(
    gdpath,
    "bounding_box_train",
)
generate_data(train_set, save_path=gdsave_path)

gdsave_path = os.path.join(
    gdpath,
    "query",
)
generate_data(query_set, save_path=gdsave_path)
gdsave_path = os.path.join(
    gdpath,
    "bounding_box_test",
)
generate_data(gallery_set, save_path=gdsave_path)

100%|████████████████████████████████████| 15913/15913 [00:22<00:00, 697.43it/s]
